In [ ]:
import surrogate_model
import argparse
import numpy as np
import pandas as pd
import src.dynamic_pipeline as dynamic_pipeline
import os
import search_model
from src.datasets.data_loader import load_data

In [2]:
def concrete():
    df = pd.read_csv('./data/concrete.csv')

    target = 'strength'
    user_request_target = 40.0

    controll_name = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age']
    importance={'cement': 1,
                    'water': 2,
    }
    optimize={'cement': 'minimize',
                'water': 'minimize',
    }
    return df, target, user_request_target, controll_name, importance, optimize 

In [3]:
def dvm():
    df = pd.read_csv('./data/dvm_merged_df.csv')

    target = 'Annual_revenue'
    user_request_target = 2485797880

    controll_name = ['Entry_price', 'Fuel_type', 'Bodytype', 'Top_speed', 'Gearbox']
    # controll_name = [col for col in df.columns if col != target]
    importance={'Entry_price': 1,
                    'Top_speed': 2,
    }
    optimize={'Entry_price': 'minimize',
                'Top_speed': 'minimize',
    }
    return df, target, user_request_target, controll_name, importance, optimize 

In [4]:
def ecommerce():
    df = pd.read_csv('./data/ecommerce_sampled_10000.csv')

    target = 'event_type_index'
    user_request_target = 3 # 구입

    controll_name = ['discount_rate']
    # controll_name = [col for col in df.columns if col != target]
    importance={'discount_rate ': 1,
    }
    optimize={'discount_rate': 'minimize',
    }
    return df, target, user_request_target, controll_name, importance, optimize 

In [5]:
def melb():
    df = pd.read_csv('./data/melb_data.csv')

    target = 'Price'
    user_request_target = 7000000

    controll_name = ['Suburb', 'Rooms', 'Landsize']
    # controll_name = [col for col in df.columns if col != target]
    importance={'Landsize': 1,
            'Rooms': 2,
    }
    optimize={'Landsize': 'minimize',
            'Rooms': 'minimize',
    }
    return df, target, user_request_target, controll_name, importance, optimize 

In [6]:
def employee():
    df = pd.read_csv('./data/employee_attrition_dataset.csv')

    target = 'Attrition'
    user_request_target = 1

    controll_name = ['Job_Role', 'Hourly_Rate', 'Average_Hours_Worked_Per_Week', 'Relationship_with_Manager', 'Job_Involvement', 'Number_of_Companies_Worked']
    # controll_name = [col for col in df.columns if col != target]
    importance={'Hourly_Rate': 1,
                'Average_Hours_Worked_Per_Week': 2,
    }
    optimize={'Hourly_Rate': 'minimize',
            'Average_Hours_Worked_Per_Week': 'maximize',
    }
    return df, target, user_request_target, controll_name, importance, optimize 

In [7]:
df, target, user_request_target, controll_name, importance, optimize = concrete() # concrete(), dvm(), ecommerce(), melb(), employee()
model = 'catboost' # 'catboost','tabpfn'

In [ ]:
ret_dict = dynamic_pipeline.preprocess_dynamic(df)
ret_dict['processed_df'].to_csv('./data/temp.csv', index=False)

In [ ]:
if target not in ret_dict['processed_df'].columns: raise Exception('unexpected target')
args = argparse.Namespace(
    target = [target],
    data_path='./data/temp.csv',
    model=model,
    prj_id = 40,
    seed=40
)
surrogate_model.main(args, ret_dict['scalers'])

In [57]:
if target not in df.columns: raise Exception('unexpected target')
controll_range = df[controll_name]
controll_range = {col: (df[col].min(), df[col].max()) for col in controll_range}

In [ ]:
controll_range

In [ ]:
print(target, df[target].min(),'~',df[target].max())

In [60]:
df_scaled = load_data(args.data_path)
X = df_scaled.drop(columns=args.target)
y = df_scaled[args.target]
x_col_list = X.columns.tolist()

X_train,y_train = X.to_numpy(),y.to_numpy()


In [61]:
scalers = ret_dict['scalers']

In [62]:
# user_req = scalers['strength'].transform(np.array(args.user_request_target[0]).reshape(-1,1))

In [63]:
ur = []
ur.append(np.array(40.0).reshape(-1,1)[0])
ur = np.array(ur)

In [ ]:
ur

In [65]:
x,y = search_model.find_top_k_similar_with_user_request(ur,X_train,y_train,k=50)

In [ ]:
x.shape
y.shape

In [67]:
for l in [controll_name, importance, optimize]:
    for c in l:
        if c not in ret_dict['processed_df'].columns:
            raise Exception(f'{c} is unexpected col_name')
args = argparse.Namespace(
    model=model,
    search_model='k_means',
    data_path='./data/temp.csv',
    controll_name=controll_name,
    controll_range= controll_range,
    target=[target],
    importance=importance,
    optimize=optimize,
    prj_id=40,
    seed=40,
    user_request_target=[user_request_target]
)

In [ ]:
args.controll_range

In [ ]:
# %%time 
x_opt = search_model.main(args,ret_dict['scalers'])

In [ ]:
df

In [ ]:
x_opt

In [ ]:
x_opt.iloc[1]